In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

def compute_youngs_modulus(
    csv_path,
    gauge_length,
    cross_section_area,
    load_col='Force',
    disp_col='Displacement',
    elastic_strain_max=0.005
):
    """
    Computes Young's modulus from tensile test data.

    Parameters
    ----------
    csv_path : str
        Path to CSV file containing load/displacement data.
    gauge_length : float
        Original specimen length (m).
    cross_section_area : float
        Cross-sectional area (m^2).
    load_col : str
        Column name for load/force.
    disp_col : str
        Column name for displacement/extension.
    elastic_strain_max : float
        Maximum strain to use for the linear elastic fit (default = 0.5%).

    Returns
    -------
    E : float
        Estimated Young’s modulus (Pa).
    stress : ndarray
    strain : ndarray
        Arrays of engineering stress and strain.
    """

    # Load file
    df = pd.read_csv(csv_path)

    # Extract data
    force = df[load_col].values
    disp  = df[disp_col].values

    # Engineering strain
    strain = disp / gauge_length

    # Engineering stress
    stress = force / cross_section_area

    # Select elastic region
    mask = strain <= elastic_strain_max
    strain_elastic = strain[mask].reshape(-1, 1)
    stress_elastic = stress[mask]

    # Linear regression (slope = E)
    model = LinearRegression().fit(strain_elastic, stress_elastic)
    E = model.coef_[0]

    return E, stress, strain


In [ ]:
def youngs_modulus(strain, stress, max_strain=0.01):
    """
    Computes Young's modulus by fitting a line to the elastic region.
    max_strain: upper bound for linear region (default = 1% strain)
    """
    mask = strain <= max_strain
    coeffs = np.polyfit(strain[mask], stress[mask], 1)
    E = coeffs[0]  # slope
    return E

def ultimate_tensile_strength(stress):
    """
    Maximum stress achieved in the test.
    """
    return np.max(stress)

def offset_yield_strength(strain, stress, offset=0.02):
    """
    Computes the offset yield strength (default 2%).
    Uses interpolation to find intersection of stress-strain curve with
    offset line of slope equal to the elastic modulus.
    """
    E = youngs_modulus(strain, stress)
    offset_line = E*(strain - offset)

    # difference between actual curve and offset line
    diff = stress - offset_line
    
    # find zero crossing
    idx = np.where(np.diff(np.sign(diff)))[0]
    if len(idx) == 0:
        return None  # No yield point found
    
    i = idx[0]
    # linear interpolation for better accuracy
    x1, x2 = strain[i], strain[i+1]
    y1, y2 = diff[i], diff[i+1]
    
    strain_yield = x1 - y1*(x2 - x1)/(y2 - y1)
    stress_yield = np.interp(strain_yield, strain, stress)
    
    return stress_yield

def total_energy(strain, stress):
    """
    Integrates the stress-strain curve using the trapezoidal rule.
    Units depend on stress/strain units (e.g., MPa -> MJ/m^3).
    """
    return np.trapz(stress, strain)

def strain_at_fracture(strain):
    """
    Maximum strain recorded before failure.
    """
    return np.max(strain)

